In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

%matplotlib inline



import gensim
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


import nltk
import re

In [ ]:
tweet_train = pd.read_csv("../input/nlp-getting-started/train.csv")
tweet_test = pd.read_csv("../input/nlp-getting-started/test.csv")

tweet_train.head()

In [ ]:
tweet_train['text'][5]

In [ ]:
tweet = pd.concat([tweet_train,tweet_test],axis=0)

In [ ]:
tweet.shape,tweet_train.shape,tweet_test.shape

In [ ]:
sentances = [word for word in tweet_train['text']]

sen = []
for word in sentances:
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', word, flags=re.MULTILINE)
    text = re.sub(r'http?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'[#,!,;,*,@,\n,),?,(,.]', '', text)
    text = text.lower()
    emoji=re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji.sub('',text)
    sen .append(text)
    
    

## Gloval Vector for word representation

In [ ]:
glove_dict = {}
with open('../input/glove-global-vectors-for-word-representation/glove.6B.100d.txt') as f:
    for line in f:
        line = line.split()
        word = line[0]
        values=np.asarray(line[1:],'float32')
        glove_dict[word]=values
len(glove_dict)

In [ ]:
glove_dict['the']

In [ ]:
corpus = []
for word in sen:
    word = word.split()
    corpus.append(word)
    
    

In [ ]:
corpus[1]

In [ ]:
MAX_LEN=50
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(corpus)
sequences=tokenizer_obj.texts_to_sequences(corpus)

tweet_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')

In [ ]:
tweet_pad

In [ ]:
word_index=tokenizer_obj.word_index
print('Number of unique words:',len(word_index))

In [ ]:
from tqdm import tqdm


In [ ]:
num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,100))

for word,i in tqdm(word_index.items()):
    if i > num_words:
        continue
    
    emb_vec=glove_dict.get(word)
    if emb_vec is not None:
        embedding_matrix[i]=emb_vec
            

In [ ]:
embedding_matrix

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from tensorflow.keras.initializers import Constant
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam



### Model building

In [ ]:
model=Sequential()

embedding=Embedding(num_words,100,embeddings_initializer=Constant(embedding_matrix),
                   input_length=MAX_LEN,trainable=False)

model.add(embedding)
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


optimzer=Adam(learning_rate=1e-5)

model.compile(loss='binary_crossentropy',optimizer=optimzer,metrics=['accuracy'])



In [ ]:
model.summary()

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(tweet_pad,tweet_train['target'], random_state=0, test_size=0.2)


In [ ]:
history=model.fit(X_train,y_train,batch_size=4,epochs=15,validation_data=(X_test,y_test),verbose=2)